In [303]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, auc, roc_curve, matthews_corrcoef
import argparse
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [304]:
def Create_Train_Test(df):
    """
    Créer les dataframes de test et d'entraînement 
    """
    y = df.pop('income')
    y = pd.get_dummies(y)['>50K']
    X = pd.get_dummies(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0, stratify=y)

    return X_train, X_test, y_train, y_test

In [305]:
def Create_X_nouvelle_data():
    """
    """
    def drop_columns(df):
        """
        Drop les colonnes fnlwgt, educational-nul car inutiles
        """
        df.drop(["fnlwgt", "educational-num"], inplace=True, axis=1)
        return df
    
    def get_opti_classes(df):
        """
        Applique les classes optimisées générées par le fichier optimisation_classes.py
        """

        def classifier(classes, var):
            """
            """
            # Parcourir chaque intervalle dans la liste
            for interval in classes:
                # Extraire l'intervalle sous forme de chaîne
                interval_str = interval[0]

                # Extraire les bornes inférieure et supérieure en enlevant les parenthèses et crochets
                lower_bound, upper_bound = interval_str[1:-1].split(', ')
                
                # Convertir les bornes en float
                lower_bound = float(lower_bound)
                upper_bound = float(upper_bound)

                # Vérifier si l'âge est dans cet intervalle
                if lower_bound < var <= upper_bound:
                    return interval_str
                
        serie_age = pd.read_csv('files/classes_opti/classes_opti_alexander_age.csv')
        serie_hpw = pd.read_csv('files/classes_opti/classes_opti_alexander_hours-per-week.csv')
        
        for serie in [serie_age, serie_hpw]:
            classes = serie.value_counts().index
            df_classes = df[serie.columns[0]].apply(lambda x : classifier(classes, x))
            df.drop(serie.columns[0], inplace=True, axis=1)
            df = pd.concat([df, df_classes], axis=1)

        return df
    
    def regroupement_V2(df):
        """
        """
        # Regroupement des personnes d'ethnies autre que blanc et noir en "Other_race"
        df['race'] = df['race'].replace(
            {'Asian-Pac-Islander|Amer-Indian-Eskimo|Other': 'Other_race'}, regex=True)

        # Regroupement des personnes d'origines différentes en une cartégorie "Other"
        df['native-country'] = df['native-country'].replace(
            {'^(?!United-States$).+': 'Other_country'}, regex=True)

        # Regroupement des personnes mariés mais avec un conjoint dans l'armé avec les personnes mariés mais avec un conjoint absent pour diverse raisons
        df['marital-status'] = df['marital-status'].replace(
            {'Married-AF-spouse|Divorced|Widowed|Separated|Married-spouse-absent': 'Alone'}, regex = True)

        df['education'] = df['education'].replace(
            {'Preschool|1st-4th|5th-6th|7th-8th|9th|10th|11th|12th': 'Low-education',
            'Doctorate|Prof-school': "Graduation",
            "Assoc-acdm|Assoc-voc" : 'Assoc'}, regex=True)

        df['workclass'] = df['workclass'].replace(
            {'Local-gov|State-gov': 'State-Local-gov',
            'Never-worked|Without-pay|\?' : 'No_income_or_unknown'}, regex=True)

        df['occupation'] = df['occupation'].replace(
            {'\?|Priv-house-serv|Other-service|Handlers-cleaners': 'Occupation:Very-Low-income',
            'Farming-fishing|Machine-op-inspct|Adm-clerical': "Occupation:Low-income",
            'Transport-moving|Craft-repair' : 'Occupation:Mid-income',
            'Sales|Armed-Forces|Tech-support|Protective-serv|Protective-serv' :'Occupation:High-income',
            'Prof-specialty|Exec-managerial' : 'Occupation:Very-High-income'}, regex=True)
        
        return df

    df_nouvelle_data = pd.read_csv('files/nouvelle_data.csv')
    df_nouvelle_data = drop_columns(df_nouvelle_data)
    df_nouvelle_data = get_opti_classes(df_nouvelle_data)
    df_nouvelle_data = regroupement_V2(df_nouvelle_data)
    X = pd.get_dummies(df_nouvelle_data)

    return X

In [306]:
def Logistic_Regression_V3(X_train, y_train):
    """
    Crée le Logit et l'entraîne grâce aux datasets d'entraînement
    Avec les hyper-paramètres du GridSearchCV
    """
    model = LogisticRegression().fit(X_train, y_train)

    return model

def DecisionTree_V2(X_train, y_train):
    """
    Créer l'arbre de décision et l'entraîne grâce aux datasets d'entraînement
    Un seul hyper-paramètres : max_depth = 7
    """
    model = tree.DecisionTreeClassifier(max_depth=13,
                                        max_leaf_nodes=63,
                                        min_samples_leaf=20, 
                                        min_samples_split=60).fit(X_train, y_train)
    return model


In [307]:
df_V2 = pd.read_csv('files/data/data_V2.csv')
X_train, X_test, y_train, y_test = Create_Train_Test(df_V2)
model_decision_tree = DecisionTree_V2(X_train, y_train)
X_nouvelle_data = Create_X_nouvelle_data()
unique, count = np.unique(model_decision_tree.predict(X_nouvelle_data), return_counts=True)
print(unique)
print(count)
print(f"{round(count[1]/sum(count)*100, 2)}%")

[False  True]
[7493 2188]
22.6%


In [ ]:
df_V2 = pd.read_csv('files/data/data_V2.csv')
X_train, X_test, y_train, y_test = Create_Train_Test(df_V2)
model_logit = Logistic_Regression_V3(X_train, y_train)
X_nouvelle_data = Create_X_nouvelle_data()
unique, count = np.unique(model_logit.predict(X_nouvelle_data), return_counts=True)
print(unique)
print(count)
print(f"{round(count[1]/sum(count)*100, 2)}%")

/Users/alexanderlunel/Documents/LILLE/Master/MasterSIAD/M2/Scoring/Etude de cas/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[False  True]
[8416 1265]
13.07%
